# Implement models with PyTorch to allow for GPU cuda compatability

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [3]:
def train_model(input_name:str, output_name:str) -> None:
    # load data
    df = pd.read_csv(input_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to numpy
    X = np.array(X)
    y = np.array(y)

    # split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # convert data to cuda tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).cuda()
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).cuda()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).cuda()

    # create batches for the data to reduce noise
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # define model (should probably use command line arguments in the future)
    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)
            
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x

    # create an instance of the model
    model = ANN_Model(input_size=X_train.shape[1]).cuda()
    device = torch.device("cuda") # replace with "cpu" if this gives error
    model.to(device)

    # define the loss function and optimizer
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # tain the model
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        for batch_inputs, batch_targets in train_loader:
            # move batch to gpu
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            # forward pass
            outputs = model(batch_inputs)
            loss = loss_function(outputs, batch_targets.unsqueeze(1))
            
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # print status
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # print accuracy
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    predicted_labels = torch.round(torch.sigmoid(outputs))
    accuracy = (predicted_labels == y_test_tensor.unsqueeze(1)).sum().item() / len(y_test_tensor)
    print(f"Accuracy: {accuracy}")

    torch.save(model.state_dict(), output_name)

In [4]:
train_model("csvs/ml_X.csv", "torch_ml")

Epoch 1/50, Loss: 0.6506658792495728
Epoch 2/50, Loss: 0.7031166553497314
Epoch 3/50, Loss: 0.5821865200996399
Epoch 4/50, Loss: 0.7699494957923889
Epoch 5/50, Loss: 0.5022382736206055
Epoch 6/50, Loss: 0.6448841094970703
Epoch 7/50, Loss: 0.4778822660446167
Epoch 8/50, Loss: 0.3296889066696167
Epoch 9/50, Loss: 0.4374728202819824
Epoch 10/50, Loss: 0.4352182149887085
Epoch 11/50, Loss: 0.5539363026618958
Epoch 12/50, Loss: 0.43466880917549133
Epoch 13/50, Loss: 0.118678517639637
Epoch 14/50, Loss: 0.19775639474391937
Epoch 15/50, Loss: 0.14845603704452515
Epoch 16/50, Loss: 0.2377241849899292
Epoch 17/50, Loss: 0.19523674249649048
Epoch 18/50, Loss: 0.1889350265264511
Epoch 19/50, Loss: 0.2878841459751129
Epoch 20/50, Loss: 0.09280321002006531
Epoch 21/50, Loss: 0.1563805788755417
Epoch 22/50, Loss: 0.116366907954216
Epoch 23/50, Loss: 0.323963463306427
Epoch 24/50, Loss: 0.09662128239870071
Epoch 25/50, Loss: 0.0900082215666771
Epoch 26/50, Loss: 0.058107588440179825
Epoch 27/50, Los

In [5]:
train_model("csvs/ou_X.csv", "torch_ou")

Epoch 1/50, Loss: 0.6859428286552429
Epoch 2/50, Loss: 0.6848615407943726
Epoch 3/50, Loss: 0.7073330879211426
Epoch 4/50, Loss: 0.6846932172775269
Epoch 5/50, Loss: 0.6977955102920532
Epoch 6/50, Loss: 0.6857306957244873
Epoch 7/50, Loss: 0.6784059405326843
Epoch 8/50, Loss: 0.6935852766036987
Epoch 9/50, Loss: 0.7112230062484741
Epoch 10/50, Loss: 0.6786715388298035
Epoch 11/50, Loss: 0.704246461391449
Epoch 12/50, Loss: 0.6899526119232178
Epoch 13/50, Loss: 0.6960074305534363
Epoch 14/50, Loss: 0.681743323802948
Epoch 15/50, Loss: 0.6753422617912292
Epoch 16/50, Loss: 0.6885039806365967
Epoch 17/50, Loss: 0.694782018661499
Epoch 18/50, Loss: 0.6897494196891785
Epoch 19/50, Loss: 0.6929440498352051
Epoch 20/50, Loss: 0.6758865714073181
Epoch 21/50, Loss: 0.6992212533950806
Epoch 22/50, Loss: 0.7025965452194214
Epoch 23/50, Loss: 0.6997569799423218
Epoch 24/50, Loss: 0.6983305811882019
Epoch 25/50, Loss: 0.6863061785697937
Epoch 26/50, Loss: 0.6824140548706055
Epoch 27/50, Loss: 0.690

In [6]:
train_model("csvs/pl_X.csv", "torch_pl")

Epoch 1/50, Loss: 0.6943276524543762
Epoch 2/50, Loss: 0.6429947018623352
Epoch 3/50, Loss: 0.5564611554145813
Epoch 4/50, Loss: 0.6193807125091553
Epoch 5/50, Loss: 0.6512973308563232
Epoch 6/50, Loss: 0.672379195690155
Epoch 7/50, Loss: 0.6304678916931152
Epoch 8/50, Loss: 0.5965667963027954
Epoch 9/50, Loss: 0.5990224480628967
Epoch 10/50, Loss: 0.5645322203636169
Epoch 11/50, Loss: 0.5150397419929504
Epoch 12/50, Loss: 0.5771586298942566
Epoch 13/50, Loss: 0.4928891956806183
Epoch 14/50, Loss: 0.5360434055328369
Epoch 15/50, Loss: 0.29941534996032715
Epoch 16/50, Loss: 0.3453601598739624
Epoch 17/50, Loss: 0.44898325204849243
Epoch 18/50, Loss: 0.3917856216430664
Epoch 19/50, Loss: 0.3421819508075714
Epoch 20/50, Loss: 0.38645750284194946
Epoch 21/50, Loss: 0.2672553062438965
Epoch 22/50, Loss: 0.45882174372673035
Epoch 23/50, Loss: 0.25774747133255005
Epoch 24/50, Loss: 0.2754007577896118
Epoch 25/50, Loss: 0.15651488304138184
Epoch 26/50, Loss: 0.4361473619937897
Epoch 27/50, Los